In [1]:
#Лидеры Цифровой Трансформации 2021, трек 6, команда R2D2
#Код основных вычислений для веб-приложения

'''В связи с тем, что API data.mos перестал работать и сайт стал выдавать ошибку, 
было принято решение импортировать исходные данные с портала data.mos и работать с ними локально. 
Скачать и посмотреть исходные данные вы можете по этой ссылке: https://disk.yandex.ru/d/i5XbZWB-uJvFFg'''

#Выполнил Бутаков Александр



import os
import json
import mpu

import pandas as pd
import numpy as np

# словарь имен входных файлов (алиас: путь без месяца, года и расширения)
FILE_NAMES_DICT = {
    'CELL_MIGRATION_FILE_NAME': 'CMatrix_Home_Work', # перемещения людей из ячейки в ячейку
    'CELL_COORDINATES': 'cell_zid_geo', # координаты ячеек для гео
    'SCHOOLS_FILE_NAME': 'school', # школы с координатами и zid ячейки
    'BUILDINGS_UNDER_CONSTRUCTION_FILE_NAME': 'капстрой' # строящиеся школы
}

# формат входных файлов
SOURCE_FORMAT ='csv'

# имена входных файлов, загружаемых томами по месяцам и годам
HISTORICAL_FILES = [
    'CELL_MIGRATION_FILE_NAME'
]

# месяца входных файлов
MONTHS = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

# года входных файлов
YEARS = [
    '2019',
    '2020',
    '2021'
]

# флаг необходимости удаления индексной колонки из CSV файлов
drop_csv_index = True
csv_index_cols_list = [
    'Unnamed: 0',
    'Unnamed: 0.1'
]

# список типов школ, которые берем для расчетов
school_type_white_list = [
    #'организация дополнительного образования',
    #'прочее',
    #'профессиональная образовательная организация',
    #'организация дополнительного профессионального образования',
    'общеобразовательная организация',
    #'дошкольная образовательная организация',
    #'образовательная организация высшего образования'
]

# во-сколько раз количество школьников должно превышать среднее для постройки школы
k_schoolchildren_overload_limit = 2

# флаг использования минимального расстояния между ячейками и значение лимита (если принимающая ячейка ближе лимита к домашней, то школу ставить не нужно)
cell_geo_distance_use = True
cell_geo_distance_limit_km = 1

# тип среднего при расчете метрик
mean_type = 'median' # 'mean'

In [2]:
# инициализируем датафреймы для входных данных
df_cell_migration = pd.DataFrame()
df_cell_geo = pd.DataFrame()
df_schools = pd.DataFrame()
df_buildings_under_construction = pd.DataFrame()

In [3]:
# считываем данные из исторических файлов
for dict_key in [k for k in FILE_NAMES_DICT.keys() if k in HISTORICAL_FILES]:
    for month in MONTHS:
        for year in YEARS:
            file_name_full = f'{FILE_NAMES_DICT[dict_key]}_{month}_{year}.{SOURCE_FORMAT}'
            
            if os.path.exists(file_name_full):
                print(f'Start read {dict_key}: {file_name_full}')
                
                if dict_key == 'CELL_MIGRATION_FILE_NAME':
                    df_cell_migration = df_cell_migration.append(pd.read_csv(file_name_full).assign(month=month).assign(year=year), ignore_index=True)
                
                if dict_key == 'CELL_COORDINATES':
                    df_cell_geo = df_cell_geo.append(pd.read_csv(file_name_full).assign(month=month).assign(year=year), ignore_index=True)
                
                if dict_key == 'SCHOOLS_FILE_NAME':
                    df_schools = df_schools.append(pd.read_csv(file_name_full).assign(month=month).assign(year=year), ignore_index=True)
                
                if dict_key == 'BUILDINGS_UNDER_CONSTRUCTION_FILE_NAME':
                    df_buildings_under_construction = df_buildings_under_construction.append(pd.read_csv(file_name_full).assign(month=month).assign(year=year), ignore_index=True)
                
                print(f'Done {dict_key}: {file_name_full}')

Start read CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_July_2020.csv
Done CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_July_2020.csv
Start read CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_October_2020.csv
Done CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_October_2020.csv
Start read CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_November_2020.csv
Done CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_November_2020.csv
Start read CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_December_2020.csv
Done CELL_MIGRATION_FILE_NAME: CMatrix_Home_Work_December_2020.csv


In [4]:
# считываем данные из остальных файлов
for dict_key in [k for k in FILE_NAMES_DICT.keys() if k not in HISTORICAL_FILES]:
    file_name_full = f'{FILE_NAMES_DICT[dict_key]}.{SOURCE_FORMAT}'
    
    print(f'Start read {dict_key}: {file_name_full}')
    
    if dict_key == 'CELL_MIGRATION_FILE_NAME':
        df_cell_migration = pd.read_csv(file_name_full)
    
    if dict_key == 'CELL_COORDINATES':
        df_cell_geo = pd.read_csv(file_name_full)
    
    if dict_key == 'SCHOOLS_FILE_NAME':
        df_schools = pd.read_csv(file_name_full)
    
    if dict_key == 'BUILDINGS_UNDER_CONSTRUCTION_FILE_NAME':
        df_buildings_under_construction = pd.read_csv(file_name_full)
    
    print(f'Done {dict_key}: {file_name_full}')

Start read CELL_COORDINATES: cell_zid_geo.csv
Done CELL_COORDINATES: cell_zid_geo.csv
Start read SCHOOLS_FILE_NAME: school.csv
Done SCHOOLS_FILE_NAME: school.csv
Start read BUILDINGS_UNDER_CONSTRUCTION_FILE_NAME: капстрой.csv
Done BUILDINGS_UNDER_CONSTRUCTION_FILE_NAME: капстрой.csv


In [5]:
# убираем индексную колонку, если необходимо
if drop_csv_index:
    for csv_index_col in csv_index_cols_list:
        if csv_index_col in df_cell_migration.columns:
            df_cell_migration.drop(columns=[csv_index_col], inplace=True)
        
        if csv_index_col in df_cell_geo.columns:
            df_cell_geo.drop(columns=[csv_index_col], inplace=True)
        
        if csv_index_col in df_schools.columns:
            df_schools.drop(columns=[csv_index_col], inplace=True)
        
        if csv_index_col in df_buildings_under_construction.columns:
            df_buildings_under_construction.drop(columns=[csv_index_col], inplace=True)

In [6]:
# формируем df с перемещениями людей в ячейки со школами: если в обеих таблицах есть колонки месяца и года - соединяем таблицы по месяцам и годам
if set(['month', 'year']).issubset(df_cell_migration.columns) and set(['month', 'year']).issubset(df_schools.columns):
    df_cell_migration_with_schools = df_cell_migration.merge(df_schools, how='left', left_on=['work_cell_zid', 'month', 'year'], right_on=['cell_zid', 'month', 'year'])
else:
    df_cell_migration_with_schools = df_cell_migration.merge(df_schools, how='left', left_on=['work_cell_zid'], right_on=['cell_zid'])

In [ ]:
# вытаскиваем первую координату ячейки для расчетов
df_cell_geo['coordinates_0_0'] = [json.loads(l)[0][0] for l in df_cell_geo['coordinates']]
df_cell_geo['coordinates_0_1'] = [json.loads(l)[0][1] for l in df_cell_geo['coordinates']]

# добавляем в df по перемещениям координаты принимающей ячейки
df_cell_migration_with_schools = df_cell_migration_with_schools.merge(df_cell_geo, how='left', left_on=['home_cell_zid'], right_on=['cell_zid'])

# переименовываем колонки для расчетов
df_cell_migration_with_schools.rename(columns={'coordinates_0_0': 'home_cell_coordinates_0_0',
                                               'coordinates_0_1': 'home_cell_coordinates_0_1',
                                               'Столбец1': 'work_cell_coordinates_0_0',
                                               'Столбец2': 'work_cell_coordinates_0_1'}, inplace=True)

# заполняем пустые координаты 0 для корректного расчета расстояния
df_cell_migration_with_schools.fillna(value={'home_cell_coordinates_0_0': 0,
                                             'home_cell_coordinates_0_1': 0,
                                             'work_cell_coordinates_0_0': 0,
                                             'work_cell_coordinates_0_1': 0}, inplace=True)

# рассчитываем расстояние между центрами ячеек (между домашней ячейкой и школой принимающей ячейки) на изогнутой земной поверхности
df_cell_migration_with_schools['cells_haversine_distance'] = df_cell_migration_with_schools.apply(lambda x: mpu.haversine_distance((x['home_cell_coordinates_0_1'],
                                                                                                                                    x['home_cell_coordinates_0_0']),
                                                                                                                                   (x['work_cell_coordinates_0_1'],
                                                                                                                                    x['work_cell_coordinates_0_0'])), axis=1)

In [ ]:
# считаем показатели по месяцам и годам

#   - отбираем школьников ячейки с типами школ из разрешенного списка
#   - фильтруем не определенные домашние ячейки с zid -1 и -2
#   - неявно фильтруем не определенные принимающие ячейки с zid -1 и -2 при фильрации типов школ (у -1 и -2 там будет NaN)
#   - исключаем из расчета перемещение школьников внутри ячейки

# количество школьников, учащихся внутри каждой домашней ячейки
df_schoolchildren_self_load = df_cell_migration_with_schools[df_cell_migration_with_schools['OrgType'].isin(school_type_white_list) &
                                                            ~df_cell_migration_with_schools['home_cell_zid'].isin([-1, -2]) &
                                                            (df_cell_migration_with_schools['home_cell_zid'] == df_cell_migration_with_schools['work_cell_zid'])] \
    .groupby(['home_cell_zid', 'year', 'month'] if set(['month', 'year']).issubset(df_cell_migration_with_schools.columns) else ['home_cell_zid'])[['customers_cnt']].sum()

# количество уезжающих школьников для каждой домашней ячейки (с учетом значения флага cell_geo_distance_use и расстояния до принимающих ячеек)
df_schoolchildren_out_load = df_cell_migration_with_schools[df_cell_migration_with_schools['OrgType'].isin(school_type_white_list) &
                                                           ~df_cell_migration_with_schools['home_cell_zid'].isin([-1, -2]) &
                                                           (df_cell_migration_with_schools['home_cell_zid'] != df_cell_migration_with_schools['work_cell_zid']) &
                                                          ((df_cell_migration_with_schools['cells_haversine_distance'] >= cell_geo_distance_limit_km) | (not cell_geo_distance_use))] \
    .groupby(['home_cell_zid', 'year', 'month'] if set(['month', 'year']).issubset(df_cell_migration_with_schools.columns) else ['home_cell_zid'])[['customers_cnt']].sum()

# количество въезжающих школьников для каждой принимающей ячейки
df_schoolchildren_in_load = df_cell_migration_with_schools[df_cell_migration_with_schools['OrgType'].isin(school_type_white_list) &
                                                         #~df_cell_migration_with_schools['home_cell_zid'].isin([-1, -2]) &
                                                          (df_cell_migration_with_schools['home_cell_zid'] != df_cell_migration_with_schools['work_cell_zid'])] \
    .groupby(['work_cell_zid', 'year', 'month'] if set(['month', 'year']).issubset(df_cell_migration_with_schools.columns) else ['work_cell_zid'])[['customers_cnt']].sum()

# перемещаем поля группировки из индекса обратно в колонки
df_schoolchildren_self_load.reset_index(drop=False, inplace=True)
df_schoolchildren_out_load.reset_index(drop=False, inplace=True)
df_schoolchildren_in_load.reset_index(drop=False, inplace=True)

# среднее количество школьников в школе по месяцам и годам (размер ячейки 500 на 500 метров и в ячейке может быть только одна школа)
if mean_type == 'median':
    df_schoolchildren_self_load_mean = df_schoolchildren_self_load.groupby(['month', 'year'])['customers_cnt'].median().reset_index(drop=False).rename(columns={'customers_cnt': 'schoolchildren_self_load_mean'})
else:
    df_schoolchildren_self_load_mean = df_schoolchildren_self_load.groupby(['month', 'year'])['customers_cnt'].mean().reset_index(drop=False).rename(columns={'customers_cnt': 'schoolchildren_self_load_mean'})

# добавляем это значение в таблицы
df_schoolchildren_self_load = df_schoolchildren_self_load.merge(df_schoolchildren_self_load_mean, how='left', on=['month', 'year'])
df_schoolchildren_out_load = df_schoolchildren_out_load.merge(df_schoolchildren_self_load_mean, how='left', on=['month', 'year'])
df_schoolchildren_in_load = df_schoolchildren_in_load.merge(df_schoolchildren_self_load_mean, how='left', on=['month', 'year'])

# перегрузка школы внутри домашней ячейки своими учениками
df_schoolchildren_self_load['self_overload'] = df_schoolchildren_self_load['customers_cnt'] / df_schoolchildren_self_load['schoolchildren_self_load_mean']

# флаг перегрузки школы внутри домашней ячейки своими учениками
df_schoolchildren_self_load['is_self_overload'] = df_schoolchildren_self_load['self_overload'] >= k_schoolchildren_overload_limit

# превышение исходящего траффика из-за перегруженности или отсутствия школы в домашней ячейке
df_schoolchildren_out_load['out_overload'] = df_schoolchildren_out_load['customers_cnt'] / df_schoolchildren_out_load['schoolchildren_self_load_mean']

# флаг превышения исходящего траффика из-за перегруженности или отсутствия школы в домашней ячейке
df_schoolchildren_out_load['is_out_overload'] = df_schoolchildren_out_load['out_overload'] >= k_schoolchildren_overload_limit

# превышение входящего траффика из-за перегруженности школы в принимающей ячейке
df_schoolchildren_in_load['in_overload'] = df_schoolchildren_in_load['customers_cnt'] / df_schoolchildren_in_load['schoolchildren_self_load_mean']

# флаг превышения входящего траффика из-за перегруженности школы в принимающей ячейке
df_schoolchildren_in_load['is_in_overload'] = df_schoolchildren_in_load['in_overload'] >= k_schoolchildren_overload_limit

In [ ]:
# считаем средние показатели за все время

if mean_type == 'median':
    # перегрузка школы внутри домашней ячейки и флаг такой перегрузки
    df_schoolchildren_self_load_cross_mean = df_schoolchildren_self_load.groupby(['home_cell_zid'])['self_overload'].median().reset_index(drop=False).rename(columns={'self_overload': 'self_overload_cross_mean'})
    df_schoolchildren_self_load_cross_mean['is_self_overload_cross_mean'] = df_schoolchildren_self_load_cross_mean['self_overload_cross_mean'] >= k_schoolchildren_overload_limit

    # превышение исходящего траффика и флаг такого превышения
    df_schoolchildren_out_load_cross_mean = df_schoolchildren_out_load.groupby(['home_cell_zid'])['out_overload'].median().reset_index(drop=False).rename(columns={'out_overload': 'out_overload_cross_mean'})
    df_schoolchildren_out_load_cross_mean['is_out_overload_cross_mean'] = df_schoolchildren_out_load_cross_mean['out_overload_cross_mean'] >= k_schoolchildren_overload_limit

    # превышение входящего траффика и флаг такого превышения
    df_schoolchildren_in_load_cross_mean = df_schoolchildren_in_load.groupby(['work_cell_zid'])['in_overload'].median().reset_index(drop=False).rename(columns={'in_overload': 'in_overload_cross_mean'})
    df_schoolchildren_in_load_cross_mean['is_in_overload_cross_mean'] = df_schoolchildren_in_load_cross_mean['in_overload_cross_mean'] >= k_schoolchildren_overload_limit
else:
    # перегрузка школы внутри домашней ячейки и флаг такой перегрузки
    df_schoolchildren_self_load_cross_mean = df_schoolchildren_self_load.groupby(['home_cell_zid'])['self_overload'].mean().reset_index(drop=False).rename(columns={'self_overload': 'self_overload_cross_mean'})
    df_schoolchildren_self_load_cross_mean['is_self_overload_cross_mean'] = df_schoolchildren_self_load_cross_mean['self_overload_cross_mean'] >= k_schoolchildren_overload_limit

    # превышение исходящего траффика и флаг такого превышения
    df_schoolchildren_out_load_cross_mean = df_schoolchildren_out_load.groupby(['home_cell_zid'])['out_overload'].mean().reset_index(drop=False).rename(columns={'out_overload': 'out_overload_cross_mean'})
    df_schoolchildren_out_load_cross_mean['is_out_overload_cross_mean'] = df_schoolchildren_out_load_cross_mean['out_overload_cross_mean'] >= k_schoolchildren_overload_limit

    # превышение входящего траффика и флаг такого превышения
    df_schoolchildren_in_load_cross_mean = df_schoolchildren_in_load.groupby(['work_cell_zid'])['in_overload'].mean().reset_index(drop=False).rename(columns={'in_overload': 'in_overload_cross_mean'})
    df_schoolchildren_in_load_cross_mean['is_in_overload_cross_mean'] = df_schoolchildren_in_load_cross_mean['in_overload_cross_mean'] >= k_schoolchildren_overload_limit

# добавляем усредненные данные к таблицам по месяцам и годам
df_schoolchildren_self_load = df_schoolchildren_self_load.merge(df_schoolchildren_self_load_cross_mean, how='left', on=['home_cell_zid'])
df_schoolchildren_out_load = df_schoolchildren_out_load.merge(df_schoolchildren_out_load_cross_mean, how='left', on=['home_cell_zid'])
df_schoolchildren_in_load = df_schoolchildren_in_load.merge(df_schoolchildren_in_load_cross_mean, how='left', on=['work_cell_zid'])

In [ ]:
# составляем общий список уникальных zid ячеек, месяцев и годов для датасета выходных данных
if set(['month', 'year']).issubset(df_schoolchildren_self_load.columns) or \
   set(['month', 'year']).issubset(df_schoolchildren_out_load.columns) or \
   set(['month', 'year']).issubset(df_schoolchildren_in_load.columns):
    uniq_cells_months_years = df_schoolchildren_self_load.rename(columns={'home_cell_zid': 'cell_zid'})[['cell_zid', 'month', 'year']] \
                                    .append(df_schoolchildren_out_load.rename(columns={'home_cell_zid': 'cell_zid'})[['cell_zid', 'month', 'year']], ignore_index=True) \
                                    .append(df_schoolchildren_in_load.rename(columns={'work_cell_zid': 'cell_zid'})[['cell_zid', 'month', 'year']], ignore_index=True) \
                                    .drop_duplicates() \
                                    .reset_index(drop=True)
else:
    uniq_cells_months_years = df_schoolchildren_self_load.rename(columns={'home_cell_zid': 'cell_zid'})[['cell_zid']] \
                                    .append(df_schoolchildren_out_load.rename(columns={'home_cell_zid': 'cell_zid'})[['cell_zid']], ignore_index=True) \
                                    .append(df_schoolchildren_in_load.rename(columns={'work_cell_zid': 'cell_zid'})[['cell_zid']], ignore_index=True) \
                                    .drop_duplicates() \
                                    .reset_index(drop=True)

# составляем датасет выходных данных
df_total = pd.DataFrame(data=uniq_cells_months_years, columns=['cell_zid', 'month', 'year'])

if set(['month', 'year']).issubset(df_schoolchildren_self_load.columns):
    df_total = df_total.merge(df_schoolchildren_self_load, how='left', left_on=['cell_zid', 'month', 'year'], right_on=['home_cell_zid', 'month', 'year'], suffixes=('', '_self'))
else:
    df_total = df_total.merge(df_schoolchildren_self_load, how='left', left_on=['cell_zid'], right_on=['home_cell_zid'], suffixes=('', '_self'))

if set(['month', 'year']).issubset(df_schoolchildren_out_load.columns):
    df_total = df_total.merge(df_schoolchildren_out_load, how='left', left_on=['cell_zid', 'month', 'year'], right_on=['home_cell_zid', 'month', 'year'], suffixes=('', '_out'))
else:
    df_total = df_total.merge(df_schoolchildren_out_load, how='left', left_on=['cell_zid'], right_on=['home_cell_zid'], suffixes=('', '_out'))

if set(['month', 'year']).issubset(df_schoolchildren_in_load.columns):
    df_total = df_total.merge(df_schoolchildren_in_load, how='left', left_on=['cell_zid', 'month', 'year'], right_on=['work_cell_zid', 'month', 'year'], suffixes=('', '_in'))
else:
    df_total = df_total.merge(df_schoolchildren_in_load, how='left', left_on=['cell_zid'], right_on=['work_cell_zid'], suffixes=('', '_in'))

# переименовываем колонки, убираем лишние, заполняем пропуски
df_total = df_total \
                .rename(columns={'customers_cnt': 'customers_cnt_self'}) \
                .drop(columns=['home_cell_zid', 'home_cell_zid_out', 'work_cell_zid', 'schoolchildren_self_load_mean_out', 'schoolchildren_self_load_mean_in']) \
                #.fillna(0)

# меняем колонки местами
head_cols = ['cell_zid', 'month', 'year']
tail_cols = [col for col in df_total.columns if col not in head_cols and col != 'schoolchildren_self_load_mean']

df_total = df_total[head_cols + ['schoolchildren_self_load_mean'] + tail_cols]

In [ ]:
# фильтруем строящиеся объекты по типу, убираем строки с несмаппившимися zid ячеек, приводим типы
df_buildings_under_construction = df_buildings_under_construction[df_buildings_under_construction['FunctionalForNG'].str.lower().isin(school_type_white_list)]
df_buildings_under_construction = df_buildings_under_construction[~df_buildings_under_construction['cell_zid'].isnull()]
df_buildings_under_construction['cell_zid'] = df_buildings_under_construction['cell_zid'].astype('int64')

# добавляем информацию по стоящимся в ячейках объектам
df_total = df_total.merge(df_buildings_under_construction[['cell_zid', 'global_id', 'ObjectName', 'ObjectAddress']], how='left', on='cell_zid')
#df_total.fillna(value={'global_id': 0, 'ObjectName': '', 'ObjectAddress': ''}, inplace=True)

In [ ]:
# добавляем в выходной датасет координаты ячейки для отрисовки на карте
df_total = df_total.merge(df_cell_geo[['cell_zid', 'coordinates']], how='left', left_on=['cell_zid'], right_on=['cell_zid'])

In [ ]:
# пишем в csv
df_total.to_csv('cell_stats_output_v3.csv')

In [ ]:
df_total

In [ ]:
# Cтатистики посчитанны в разрезе ячеек:
#    часть статистик - с учетом месяцев и годов,
#    часть статистик сковозные - без учета месяцев и годов.

# Колонки:
#    schoolchildren_self_load_mean - среднее количество школьников внутри ячейки по месяцам и годам
#    customers_cnt - количество перемещающихся школьников
#    overload - коэффициент перегрузки ячейки школьниками
#    is_*_overload - флаг перегрузки ячейки школьниками (True при привышении overload заданного лимита)
#    global_id - id строящегося объекта в ячейке, если такой есть
#    ObjectName - название строящегося объекта
#    ObjectAddress - адрес строящегося объекта
#    coordinates - массив пар координат ячейки в json

# Суффиксы:
#    self - колонки статистики по перемещениям внутри ячейки
#    out - колонки статистики по перемещениям из ячейки
#    in - колонки статистики по перемещениям в ячейку
#    cross - колонки сковозной статистики (без учета месяца и года)